# Evaluation

In [2]:
import pandas as pd
import numpy as np

xls = pd.ExcelFile("PT_Dataset .xlsx")
df_orig = pd.read_excel(xls, "Original Data")
df_norm = pd.read_excel(xls, "Normonized")
df_rank0 = pd.read_excel(xls, "Topsis_Entropy_Ranking")
df_rank_opt = pd.read_excel(xls, "Optimized_Ranking")

df_orig.head(), df_norm.head(), df_rank0.head(), df_rank_opt.head()


(   trainer_id       name  videos_count  total_likes  workout_recommendations  \
 0           1  Trainer 1           247       289134                      154   
 1           2  Trainer 2           114       122357                       68   
 2           3  Trainer 3           482       473810                       43   
 3           4  Trainer 4           403       317531                      100   
 4           5  Trainer 5           127       116388                      141   
 
    athlete_rating  years_experience  ACE  NASM  ISSA  retention_rate  \
 0             4.2                22    1     1     1            0.63   
 1             4.9                24    0     1     1            0.57   
 2             3.2                 8    1     0     0            0.58   
 3             4.3                 3    1     1     1            0.34   
 4             4.4                 3    1     0     1            0.65   
 
    profile_completeness                   specialities available_times 

# Functional Testing

In [6]:
# Implement entropy + TOPSIS according to the user's code
features = [
    'videos_count_norm',
    'total_likes_norm',
    'workout_recommendations_norm',
    'athlete_rating_norm',
    'retention_rate_norm',
    'profile_completeness_norm',
    'cert_score'
]

X = df_norm[features].values.astype(float)

# Proportion matrix
P = X / X.sum(axis=0)

# Entropy
eps = 1e-12
E = -np.nansum(P * np.log(P + eps), axis=0) / np.log(len(X))
d = 1 - E
w = d / d.sum()

Z_plus = X.max(axis=0)
Z_minus = X.min(axis=0)

D_plus = np.sqrt(((X - Z_plus) ** 2 * w).sum(axis=1))
D_minus = np.sqrt(((X - Z_minus) ** 2 * w).sum(axis=1))
topsis_score = D_minus / (D_plus + D_minus)

df_recalc = df_norm[['trainer_id','name','Group']].copy()
df_recalc['topsis_score'] = topsis_score
df_recalc['topsis_rank'] = df_recalc['topsis_score'].rank(ascending=False, method='min').astype(int)

# Compare with provided sheet (sort by rank)
cmp = df_recalc.sort_values('topsis_rank').head(20).reset_index(drop=True)
cmp_provided = df_rank0.sort_values('topsis_rank').head(20).reset_index(drop=True)

display(cmp)
display(cmp_provided)




#display_dataframe_to_user("Recomputed_TOPSIS_Top20", cmp)
#display_dataframe_to_user("Provided_TOPSIS_Top20", cmp_provided)

# Compute top-20 overlap
overlap = set(cmp['trainer_id']).intersection(set(cmp_provided['trainer_id']))
overlap_count = len(overlap)
overlap_count


,trainer_id,name,Group,topsis_score,topsis_rank
0,230,Trainer 230,B,0.916717,1
1,840,Trainer 840,A,0.907970,2
2,195,Trainer 195,B,0.900355,3
3,916,Trainer 916,B,0.891000,4
4,285,Trainer 285,C,0.889288,5
5,998,Trainer 998,A,0.888475,6
6,375,Trainer 375,B,0.879787,7
7,157,Trainer 157,C,0.878439,8
8,579,Trainer 579,A,0.876756,9
9,487,Trainer 487,A,0.865788,10


,trainer_id,name,topsis_score,topsis_rank,Group
0,230,Trainer 230,0.917,1,B
1,840,Trainer 840,0.908,2,A
2,195,Trainer 195,0.900,3,B
3,916,Trainer 916,0.891,4,B
4,285,Trainer 285,0.889,5,C
5,998,Trainer 998,0.888,6,A
6,375,Trainer 375,0.880,7,B
7,157,Trainer 157,0.878,8,C
8,579,Trainer 579,0.877,9,A
9,487,Trainer 487,0.866,10,A


20

In [7]:
# Recompute optimized final score using provided weights
weight_dict = {
    'videos_count_norm': 0.106,
    'total_likes_norm': 0.062,
    'workout_recommendations_norm': 0.182,
    'athlete_rating_norm': 0.049,
    'retention_rate_norm': 0.228,
    'profile_completeness_norm': 0.030,
    'cert_score': 0.142,
    'bonus_points_norm': 0.200
}

final_score = sum(df_norm[k] * v for k, v in weight_dict.items())
df_final = df_norm[['trainer_id','name','Group']].copy()
df_final['final_score'] = final_score
df_final['final_rank'] = df_final['final_score'].rank(ascending=False, method='min').astype(int)

cmp_opt = df_final.sort_values('final_rank').head(20).reset_index(drop=True)
cmp_opt_provided = df_rank_opt.sort_values('final_rank').head(20).reset_index(drop=True)
display(cmp_opt)
display(cmp_opt_provided)

overlap_opt = len(set(cmp_opt['trainer_id']).intersection(set(cmp_opt_provided['trainer_id'])))
overlap_opt


,trainer_id,name,Group,final_score,final_rank
0,230,Trainer 230,B,0.955889,1
1,840,Trainer 840,A,0.920776,2
2,916,Trainer 916,B,0.906855,3
3,285,Trainer 285,C,0.906466,4
4,579,Trainer 579,A,0.901226,5
5,748,Trainer 748,A,0.891353,6
6,722,Trainer 722,C,0.873757,7
7,195,Trainer 195,B,0.871371,8
8,593,Trainer 593,A,0.868432,9
9,750,Trainer 750,B,0.860708,10


,trainer_id,name,final_score,final_rank,Group,salary_gbp_month
0,230,Trainer 230,0.956,1,B,5075
1,840,Trainer 840,0.921,2,A,3638
2,916,Trainer 916,0.907,3,B,4685
3,285,Trainer 285,0.906,4,C,4861
4,579,Trainer 579,0.901,5,A,4373
5,748,Trainer 748,0.891,6,A,4096
6,722,Trainer 722,0.874,7,C,5028
7,195,Trainer 195,0.871,8,B,5130
8,593,Trainer 593,0.868,9,A,5282
9,750,Trainer 750,0.861,10,B,4070


20

In [9]:
import numpy as np
import pandas as pd

def recompute_topsis_entropy(df_norm_local):
    features = [
        'videos_count_norm',
        'total_likes_norm',
        'workout_recommendations_norm',
        'athlete_rating_norm',
        'retention_rate_norm',
        'profile_completeness_norm',
        'cert_score'
    ]
    X = df_norm_local[features].values.astype(float)
    # proportion matrix (protect division-by-zero)
    col_sums = X.sum(axis=0)
    safe_col_sums = np.where(col_sums==0, 1.0, col_sums)
    P = X / safe_col_sums
    eps = 1e-12
    E = -np.nansum(P * np.log(P + eps), axis=0) / np.log(len(X))
    d = 1 - E
    # handle zero sum
    if d.sum()==0:
        w = np.ones_like(d) / len(d)
    else:
        w = d / d.sum()
    Z_plus = np.nanmax(X, axis=0)
    Z_minus = np.nanmin(X, axis=0)
    # replace NaN with column means for distance
    X_filled = np.where(np.isnan(X), np.nanmean(X, axis=0), X)
    D_plus = np.sqrt(((X_filled - Z_plus) ** 2 * w).sum(axis=1))
    D_minus = np.sqrt(((X_filled - Z_minus) ** 2 * w).sum(axis=1))
    denom = (D_plus + D_minus)
    denom = np.where(denom==0, 1.0, denom)
    score = D_minus / denom
    out = df_norm_local[['trainer_id','name','Group']].copy()
    out['topsis_score'] = score
    out['topsis_rank'] = out['topsis_score'].rank(ascending=False, method='min').astype(int)
    return out, w

def recompute_final(df_norm_local, weight_dict):
    score = sum(df_norm_local[k] * v for k, v in weight_dict.items())
    out = df_norm_local[['trainer_id','name','Group']].copy()
    out['final_score'] = score
    out['final_rank'] = out['final_score'].rank(ascending=False, method='min').astype(int)
    return out

# Baseline
baseline_rank, baseline_w = recompute_topsis_entropy(df_norm.copy())
baseline_final = recompute_final(df_norm.copy(), {
    'videos_count_norm': 0.106,
    'total_likes_norm': 0.062,
    'workout_recommendations_norm': 0.182,
    'athlete_rating_norm': 0.049,
    'retention_rate_norm': 0.228,
    'profile_completeness_norm': 0.030,
    'cert_score': 0.142,
    'bonus_points_norm': 0.200
})

# Fault case 1: a trainer all zeros
df_fault1 = df_norm.copy()
row_idx = df_fault1.index[0]
for col in ['videos_count_norm','total_likes_norm','workout_recommendations_norm','athlete_rating_norm','retention_rate_norm','profile_completeness_norm','cert_score']:
    df_fault1.loc[row_idx, col] = 0.0

rank_fault1, w_fault1 = recompute_topsis_entropy(df_fault1)

# Fault case 2: one group with no variance (set all feature columns in Group 'A' to their group mean)
df_fault2 = df_norm.copy()
maskA = df_fault2['Group']=='A'
for col in ['videos_count_norm','total_likes_norm','workout_recommendations_norm','athlete_rating_norm','retention_rate_norm','profile_completeness_norm','cert_score']:
    meanA = df_fault2.loc[maskA, col].mean()
    df_fault2.loc[maskA, col] = meanA

rank_fault2, w_fault2 = recompute_topsis_entropy(df_fault2)

# Fault case 3: extreme value
df_fault3 = df_norm.copy()
df_fault3.loc[row_idx, 'videos_count_norm'] = 1.0
df_fault3.loc[row_idx, 'total_likes_norm'] = 1.0
rank_fault3, w_fault3 = recompute_topsis_entropy(df_fault3)

# Fault case 4: introduce NaNs
df_fault4 = df_norm.copy()
for col in ['videos_count_norm','total_likes_norm','workout_recommendations_norm','athlete_rating_norm','retention_rate_norm','profile_completeness_norm','cert_score']:
    df_fault4.loc[row_idx, col] = np.nan

rank_fault4, w_fault4 = recompute_topsis_entropy(df_fault4)

# Compare ranks for affected trainer
trainer_id_affected = int(df_norm.loc[row_idx, 'trainer_id'])

def get_rank(df_rank, tid):
    return int(df_rank.loc[df_rank['trainer_id']==tid, ['topsis_rank']].values[0])

results = pd.DataFrame({
    'case': ['baseline','all_zero','group_no_variance','extreme_values','NaNs'],
    'rank_of_affected_trainer': [
        get_rank(baseline_rank, trainer_id_affected),
        get_rank(rank_fault1, trainer_id_affected),
        get_rank(rank_fault2, trainer_id_affected),
        get_rank(rank_fault3, trainer_id_affected),
        get_rank(rank_fault4, trainer_id_affected),
    ]
})
display(results)

# Reproducibility: run twice and compare hashes
rank1,_ = recompute_topsis_entropy(df_norm.copy())
rank2,_ = recompute_topsis_entropy(df_norm.copy())
repro_ok = rank1['topsis_rank'].equals(rank2['topsis_rank'])

repro_ok


C:\Users\97059\AppData\Local\Temp\ipykernel_38716\3754630533.py:95: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(df_rank.loc[df_rank['trainer_id']==tid, ['topsis_rank']].values[0])
C:\Users\97059\AppData\Local\Temp\ipykernel_38716\3754630533.py:95: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(df_rank.loc[df_rank['trainer_id']==tid, ['topsis_rank']].values[0])
C:\Users\97059\AppData\Local\Temp\ipykernel_38716\3754630533.py:95: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operati

,case,rank_of_affected_trainer
0,baseline,33
1,all_zero,1000
2,group_no_variance,301
3,extreme_values,23
4,NaNs,407


True

In [10]:
def recompute_bonus(df):
    feats = [
        'videos_count_norm',
        'total_likes_norm',
        'workout_recommendations_norm',
        'athlete_rating_norm',
        'retention_rate_norm',
        'profile_completeness_norm',
        'cert_score'
    ]
    def calc_bonus(group_df):
        arr = group_df[feats].values
        q30 = group_df[feats].quantile(0.3)
        q70 = group_df[feats].quantile(0.7)
        # +1 if > q70, -1 if < q30 across features (sum per row)
        gt70 = (arr > q70.values).astype(int)
        lt30 = (arr < q30.values).astype(int)
        bonus = gt70.sum(axis=1) - lt30.sum(axis=1)
        # +5 if each feature >= median
        q50 = group_df[feats].quantile(0.5)
        ge50_all = (arr >= q50.values).all(axis=1).astype(int)
        bonus = bonus + ge50_all * 5
        return pd.Series(bonus, index=group_df.index)
    out = df.copy()
    out['bonus_points_recalc'] = out.groupby('Group', group_keys=False).apply(calc_bonus)
    # Normalize
    bmin, bmax = out['bonus_points_recalc'].min(), out['bonus_points_recalc'].max()
    out['bonus_points_norm_recalc'] = (out['bonus_points_recalc'] - bmin) / (bmax - bmin) if bmax!=bmin else 0.0
    return out

# Apply on fault2 (no variance in Group A)
df_bonus_fault2 = recompute_bonus(df_fault2.copy())
# Summarize variance per group
var_by_group = df_bonus_fault2.groupby('Group')['bonus_points_recalc'].agg(['min','max','nunique'])
display(var_by_group.reset_index())


C:\Users\97059\AppData\Local\Temp\ipykernel_38716\2925667406.py:25: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  out['bonus_points_recalc'] = out.groupby('Group', group_keys=False).apply(calc_bonus)


,Group,min,max,nunique
0,A,5,5,1
1,B,-5,10,14
2,C,-6,9,13


In [12]:
import numpy as np
import pandas as pd

weights_df = pd.DataFrame({
    'feature': [
        'videos_count_norm','total_likes_norm','workout_recommendations_norm',
        'athlete_rating_norm','retention_rate_norm','profile_completeness_norm','cert_score'
    ],
    'baseline_w': baseline_w,
    'extreme_values_w': w_fault3
})
weights_df['delta'] = weights_df['extreme_values_w'] - weights_df['baseline_w']
display(weights_df)


,feature,baseline_w,extreme_values_w,delta
0,videos_count_norm,0.131974,0.132027,0.000052
1,total_likes_norm,0.077772,0.077852,0.000080
2,workout_recommendations_norm,0.228367,0.228329,-0.000038
3,athlete_rating_norm,0.061445,0.061434,-0.000010
4,retention_rate_norm,0.285357,0.285309,-0.000048
5,profile_completeness_norm,0.037598,0.037592,-0.000006
6,cert_score,0.177487,0.177457,-0.000030


# Behavioural Testing

In [3]:
# === Behavioural Testing: consolidated with only display() ===

import pandas as pd
import numpy as np
from IPython.display import display

# ---------- (1) Load data ----------
xls = pd.ExcelFile("PT_Dataset .xlsx")
df_norm = pd.read_excel(xls, "Normonized") 
df_rank0 = pd.read_excel(xls, "Topsis_Entropy_Ranking")
df_rank_opt = pd.read_excel(xls, "Optimized_Ranking")

# ---------- (2) Configure features and weights ----------
features = [
    'videos_count_norm',
    'total_likes_norm',
    'workout_recommendations_norm',
    'athlete_rating_norm',
    'retention_rate_norm',
    'profile_completeness_norm',
    'cert_score',
    'bonus_points_norm'
]

weight = {
    'videos_count_norm': 0.106,
    'total_likes_norm': 0.062,
    'workout_recommendations_norm': 0.182,
    'athlete_rating_norm': 0.049,
    'retention_rate_norm': 0.228,
    'profile_completeness_norm': 0.030,
    'cert_score': 0.142,
    'bonus_points_norm': 0.200
}

# ---------- (3) Merge data ----------
df = df_norm.merge(df_rank0, on='trainer_id', how='left', suffixes=('', '_rank0'))
df = df.merge(df_rank_opt, on='trainer_id', how='left', suffixes=('_x', '_y'))

# ---------- (4) Unify Group column ----------
if 'Group' not in df.columns:
    gx = 'Group_x' if 'Group_x' in df.columns else None
    gy = 'Group_y' if 'Group_y' in df.columns else None
    if gx or gy:
        df['Group'] = df.get(gy, pd.Series(index=df.index)).combine_first(
            df.get(gx, pd.Series(index=df.index))
        )
    else:
        df['Group'] = 'Unknown'

# ---------- (5) Top-N vs Bottom-N comparison ----------
def compare_top_bottom(df_in, rank_col, N=20):
    cols = [c for c in features[:-1] if c in df_in.columns]
    top = df_in.nsmallest(N, rank_col)
    bot = df_in.nlargest(N, rank_col)
    top_avg = top[cols].mean().rename(f'Top{N}_avg')
    bot_avg = bot[cols].mean().rename(f'Bottom{N}_avg')
    comp = pd.concat([top_avg, bot_avg], axis=1)
    comp['lift(Top-Bottom)'] = comp.iloc[:, 0] - comp.iloc[:, 1]
    return comp

comp_final = compare_top_bottom(df, 'final_rank', N=20)
display(comp_final.reset_index().rename(columns={'index':'feature'}))

# ---------- (6) Ranking changes ----------
df['delta_rank'] = df['topsis_rank'] - df['final_rank']
cols_show = [c for c in ['trainer_id','name','Group','topsis_rank','final_rank','delta_rank','bonus_points_norm'] if c in df.columns]

movers_up = df.sort_values('delta_rank', ascending=False).head(20)[cols_show]
movers_down = df.sort_values('delta_rank', ascending=True).head(20)[cols_show]

display(movers_up.reset_index(drop=True))
display(movers_down.reset_index(drop=True))

significant_change = (df['delta_rank'].abs() >= 50).mean()
total_changed = (df['delta_rank'] != 0).mean()

# ---------- (7) Feature contribution and correlation ----------
for k, v in weight.items():
    if k not in df.columns:
        df[k] = 0.0
    df[f'contrib_{k}'] = df[k] * v

contrib_cols = [c for c in df.columns if c.startswith('contrib_')]
corr_records = []
for c in contrib_cols:
    feat = c.replace('contrib_', '')
    val = df[feat].corr(df['final_score']) if feat in df.columns else np.nan
    corr_records.append((feat, val))

corr_df = pd.DataFrame(corr_records, columns=['feature', 'pearson_corr_with_final']).sort_values(
    'pearson_corr_with_final', ascending=False
)
display(corr_df)

# ---------- (8) Fairness analysis ----------
def group_share_in_topK(df_in, rank_col, K):
    topk = df_in.nsmallest(K, rank_col)
    return topk['Group'].value_counts(normalize=True).rename(f'top{K}_share')

group_shares = pd.concat([
    group_share_in_topK(df, 'topsis_rank', 50),
    group_share_in_topK(df, 'final_rank', 50),
    group_share_in_topK(df, 'topsis_rank', 100),
    group_share_in_topK(df, 'final_rank', 100),
], axis=1).fillna(0.0)
display(group_shares.reset_index().rename(columns={'index':'Group'}))

# ---------- (9) Bonus contribution ----------
df['contrib_sum'] = df[contrib_cols].sum(axis=1).replace(0, np.nan)
df['bonus_contrib'] = df['bonus_points_norm'] * weight['bonus_points_norm'] if 'bonus_points_norm' in df.columns else 0.0
df['bonus_contrib_share'] = (df['bonus_contrib'] / df['contrib_sum']).fillna(0.0)

bonus_summary = pd.DataFrame({
    'set': ['Top50','Bottom50','All'],
    'bonus_share_mean': [
        df.nsmallest(50,'final_rank')['bonus_contrib_share'].mean(),
        df.nlargest(50,'final_rank')['bonus_contrib_share'].mean(),
        df['bonus_contrib_share'].mean()
    ]
})
display(bonus_summary)

# ---------- (10) Summary metrics (flattened) ----------
ret_pair = (
    float(comp_final.loc['retention_rate_norm','Top20_avg']),
    float(comp_final.loc['retention_rate_norm','Bottom20_avg'])
) if 'retention_rate_norm' in comp_final.index else (np.nan, np.nan)

cert_pair = (
    float(comp_final.loc['cert_score','Top20_avg']),
    float(comp_final.loc['cert_score','Bottom20_avg'])
) if 'cert_score' in comp_final.index else (np.nan, np.nan)

summary_flat = {
    "significant_change_prop(|delta_rank|>=50)": float(significant_change),
    "changed_prop(delta_rank!=0)": float(total_changed),
    "Top20_retention_mean": ret_pair[0],
    "Bottom20_retention_mean": ret_pair[1],
    "Top20_cert_mean": cert_pair[0],
    "Bottom20_cert_mean": cert_pair[1],
}

summary_df = pd.DataFrame([summary_flat])
display(summary_df)

# ---------- (11) Key findings ----------
corr_top3 = corr_df.dropna(subset=['pearson_corr_with_final']).head(3)
corr_top3_values = list(
    zip(corr_top3['feature'].tolist(), corr_top3['pearson_corr_with_final'].round(3).tolist())
)

gs = group_shares.copy()
gs.columns = ['topsis_top50','final_top50','topsis_top100','final_top100']
group_shares_out = gs.round(3).reset_index()

artifacts = {
    "summary_stats": summary_stats,
    "corr_top3": corr_top3_values,
    "group_shares": group_shares_out.to_dict(orient='records'),
    "bonus_summary": bonus_summary.round(4).to_dict(orient='records')
}
display(artifacts)


,feature,Top20_avg,Bottom20_avg,lift(Top-Bottom)
0,videos_count_norm,0.94610,0.63410,0.31200
1,total_likes_norm,0.86915,0.60385,0.26530
2,workout_recommendations_norm,0.91635,0.29140,0.62495
3,athlete_rating_norm,0.86125,0.66750,0.19375
4,retention_rate_norm,0.76920,0.23840,0.53080
5,profile_completeness_norm,0.97000,0.92000,0.05000
6,cert_score,0.98500,0.54500,0.44000


,trainer_id,Group,topsis_rank,final_rank,delta_rank,bonus_points_norm
0,225,B,462,91,371,0.631579
1,277,C,654,337,317,0.578947
2,258,C,679,373,306,0.526316
3,664,B,517,231,286,0.578947
4,764,B,488,222,266,0.578947
5,145,B,491,228,263,0.578947
6,605,B,346,84,262,0.631579
7,160,C,463,212,251,0.526316
8,546,A,615,368,247,0.421053
9,279,B,677,432,245,0.578947


,trainer_id,Group,topsis_rank,final_rank,delta_rank,bonus_points_norm
0,377,A,344,613,-269,0.210526
1,666,A,497,757,-260,0.157895
2,381,A,267,504,-237,0.263158
3,91,C,152,383,-231,0.263158
4,560,B,281,509,-228,0.263158
5,227,A,166,394,-228,0.263158
6,337,B,328,551,-223,0.210526
7,850,A,355,575,-220,0.210526
8,699,A,522,741,-219,0.157895
9,632,A,247,462,-215,0.263158


,feature,pearson_corr_with_final
7,bonus_points_norm,0.849343
4,retention_rate_norm,0.534484
2,workout_recommendations_norm,0.522444
6,cert_score,0.475952
0,videos_count_norm,0.396131
1,total_likes_norm,0.392147
3,athlete_rating_norm,0.157494
5,profile_completeness_norm,0.064227


,Group,top50_share,top50_share,top100_share,top100_share
0,A,0.36,0.36,0.36,0.34
1,B,0.32,0.34,0.30,0.34
2,C,0.32,0.30,0.34,0.32


,set,bonus_share_mean
0,Top50,0.160409
1,Bottom50,0.080050
2,All,0.116455


,significant_change_prop(|delta_rank|>=50),changed_prop(delta_rank!=0),Top20_retention_mean,Bottom20_retention_mean,Top20_cert_mean,Bottom20_cert_mean
0,0.516,0.987,0.7692,0.2384,0.985,0.545


{'summary_stats': {'significant_change_prop(|delta_rank|>=50)': 0.516,
  'changed_prop(delta_rank!=0)': 0.987,
  'top20_means_retention_vs_bottom20': (0.7692, 0.2384),
  'top20_means_cert_vs_bottom20': (0.985, 0.545)},
 'corr_top3': [('bonus_points_norm', 0.849),
  ('retention_rate_norm', 0.534),
  ('workout_recommendations_norm', 0.522)],
 'group_shares': [{'Group': 'A',
   'topsis_top50': 0.36,
   'final_top50': 0.36,
   'topsis_top100': 0.36,
   'final_top100': 0.34},
  {'Group': 'B',
   'topsis_top50': 0.32,
   'final_top50': 0.34,
   'topsis_top100': 0.3,
   'final_top100': 0.34},
  {'Group': 'C',
   'topsis_top50': 0.32,
   'final_top50': 0.3,
   'topsis_top100': 0.34,
   'final_top100': 0.32}],
 'bonus_summary': [{'set': 'Top50', 'bonus_share_mean': 0.1604},
  {'set': 'Bottom50', 'bonus_share_mean': 0.0801},
  {'set': 'All', 'bonus_share_mean': 0.1165}]}